In [2]:
import pandas as pd
import numpy as np

import warnings
warnings.simplefilter(action='ignore')

In [ ]:
import ssl

try:
   _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    # Legacy Python that doesn't verify HTTPS certificates by default
    pass
else:
    # Handle target environment that doesn't support HTTPS verification
    ssl._create_default_https_context = _create_unverified_https_context

In [9]:
# statistical imputation transform for the horse colic dataset
from numpy import isnan
from pandas import read_csv
from sklearn.impute import SimpleImputer

In [46]:
# load dataset
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/horse-colic.csv'
dataframe = read_csv(url, header=None, na_values='?')
dataframe

0   1        2     3      4     5    6    7    8    9   ...    18    19  \
0    2.0   1   530101  38.5   66.0  28.0  3.0  3.0  NaN  2.0  ...  45.0   8.4   
1    1.0   1   534817  39.2   88.0  20.0  NaN  NaN  4.0  1.0  ...  50.0  85.0   
2    2.0   1   530334  38.3   40.0  24.0  1.0  1.0  3.0  1.0  ...  33.0   6.7   
3    1.0   9  5290409  39.1  164.0  84.0  4.0  1.0  6.0  2.0  ...  48.0   7.2   
4    2.0   1   530255  37.3  104.0  35.0  NaN  NaN  6.0  2.0  ...  74.0   7.4   
..   ...  ..      ...   ...    ...   ...  ...  ...  ...  ...  ...   ...   ...   
295  1.0   1   533886   NaN  120.0  70.0  4.0  NaN  4.0  2.0  ...  55.0  65.0   
296  2.0   1   527702  37.2   72.0  24.0  3.0  2.0  4.0  2.0  ...  44.0   NaN   
297  1.0   1   529386  37.5   72.0  30.0  4.0  3.0  4.0  1.0  ...  60.0   6.8   
298  1.0   1   530612  36.5  100.0  24.0  3.0  3.0  3.0  1.0  ...  50.0   6.0   
299  1.0   1   534618  37.2   40.0  20.0  NaN  NaN  NaN  NaN  ...  36.0  62.0   

      20   21   22  23     24  25  26  27  
0    NaN  NaN  2.0   2  11300   0   0   2  
1    2.0  2.0  3.0   2   2208   0   0   2  
2    NaN  NaN  1.0   2      0   0   0   1  
3    3.0  5.3  2.0   1   2208   0   0   1  
4    NaN  NaN  2.0   2   4300   0   0   2  
..   ...  ...  ...  ..    ...  ..  ..  ..  
295  NaN  NaN  3.0   2   3205   0   0   2  
296  3.0  3.3  3.0   1   2208   0   0   1  
297  NaN  NaN  2.0   1   3205   0   0   2  
298  3.0  3.4  1.0   1   2208   0   0   1  
299  1.0  1.0  3.0   2   6112   0   0   2  

[300 rows x 28 columns]

In [45]:
# split into input and output elements
data = dataframe.values
ix = [i for i in range(data.shape[1]) if i != 23]
X, y = data[:, ix], data[:, 23]

print('ix\n',ix[:])
print('X\n',X[:1])
print('y\n',y[:10])

ix
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 24, 25, 26, 27]
X
 [[2.00000e+00 1.00000e+00 5.30101e+05 3.85000e+01 6.60000e+01 2.80000e+01
  3.00000e+00 3.00000e+00         nan 2.00000e+00 5.00000e+00 4.00000e+00
  4.00000e+00         nan         nan         nan 3.00000e+00 5.00000e+00
  4.50000e+01 8.40000e+00         nan         nan 2.00000e+00 1.13000e+04
  0.00000e+00 0.00000e+00 2.00000e+00]]
y
 [2. 2. 2. 1. 2. 2. 1. 1. 1. 2.]


In [15]:
# print total missing
print('Missing: %d' % sum(isnan(X).flatten()))

Missing: 1605


In [22]:
# define imputer
imputer = SimpleImputer(strategy='mean')
# fit on the dataset
imputer.fit(X)
# transform the dataset
Xtrans = imputer.transform(X)
# print total missing
print('Missing: %d' % sum(isnan(Xtrans).flatten()))

Missing: 0


In [42]:
dataframe.isna().sum()

0       1
1       0
2       0
3      60
4      24
5      58
6      56
7      69
8      47
9      32
10     55
11     44
12     56
13    104
14    106
15    247
16    102
17    118
18     29
19     33
20    165
21    198
22      1
23      0
24      0
25      0
26      0
27      0
dtype: int64

In [18]:
# report which features were selected by RFE
from sklearn.datasets import make_classification
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
# define dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=5, n_redundant=5,
random_state=1)
# define RFE
rfe = RFE(estimator=DecisionTreeClassifier(), n_features_to_select=5)
# fit RFE
rfe.fit(X, y)
# summarize all features
for i in range(X.shape[1]):
    print('Column: %d, Selected=%s, Rank: %d' % (i, rfe.support_[i], rfe.ranking_[i]))

Column: 0, Selected=False, Rank: 3
Column: 1, Selected=False, Rank: 6
Column: 2, Selected=True, Rank: 1
Column: 3, Selected=True, Rank: 1
Column: 4, Selected=True, Rank: 1
Column: 5, Selected=False, Rank: 5
Column: 6, Selected=True, Rank: 1
Column: 7, Selected=False, Rank: 4
Column: 8, Selected=True, Rank: 1
Column: 9, Selected=False, Rank: 2
